In [1]:
import random
import csv
import paramiko
import logging
import time
from multiprocessing import Pool

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s [%(processName)s] %(message)s', filename='bandwidth_injection.log')

# Function to generate a bandwidth matrix for a given number of nodes
def generate_bandwidth_matrix(num_nodes, min_bandwidth=200, max_bandwidth=600):
    bandwidth_matrix = [[0 if i == j else random.randint(min_bandwidth, max_bandwidth)
                         for j in range(num_nodes)] for i in range(num_nodes)]
    return bandwidth_matrix

# Generate bandwidth matrix for 9 nodes
num_nodes = 9
bandwidth_matrix = generate_bandwidth_matrix(num_nodes, min_bandwidth=200, max_bandwidth=800)

# List of node names to map to bandwidth matrix indices
node_names = [
    'k8s-worker-1', 'k8s-worker-2', 'k8s-worker-3', 'k8s-worker-4',
    'k8s-worker-5', 'k8s-worker-6', 'k8s-worker-7', 'k8s-worker-8', 'k8s-worker-9'
]

bandwidth_matrix

[[0, 657, 580, 434, 653, 794, 333, 778, 339],
 [415, 0, 585, 699, 689, 434, 724, 252, 543],
 [522, 713, 0, 294, 211, 263, 231, 489, 721],
 [257, 708, 573, 0, 419, 460, 336, 259, 778],
 [652, 611, 291, 506, 0, 498, 518, 443, 618],
 [701, 571, 626, 704, 721, 0, 764, 497, 774],
 [650, 295, 303, 676, 536, 483, 0, 797, 355],
 [375, 503, 205, 600, 387, 413, 238, 0, 357],
 [391, 652, 378, 625, 697, 680, 552, 744, 0]]

In [ ]:

# Function to apply bandwidth settings between nodes
def apply_bandwidth_between_nodes(source_node_name, username, key_path, interface, bandwidth_matrix, node_details, node_names):
    client = paramiko.SSHClient()
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    client.load_system_host_keys()

    try:
        source_node_ip = node_details[source_node_name]['ip']
        client.connect(source_node_ip, username=username, key_filename=key_path)

        # Clear existing rules and set root qdisc
        client.exec_command(f"sudo tc qdisc del dev {interface} root")
        client.exec_command(f"sudo tc qdisc add dev {interface} root handle 1: htb default 1")
        logging.info(f'Bandwidth cleared for {source_node_name}')

        # Get the index of the source node
        source_index = node_names.index(source_node_name)

        for dst_node, details in node_details.items():
            if dst_node == source_node_name:
                continue

            dst_node_ip = details['ip']
            dst_index = node_names.index(dst_node)
            bandwidth = bandwidth_matrix[source_index][dst_index]

            logging.info(f'Setting bandwidth between {source_node_name} and {dst_node}: {bandwidth} Mbps')
            client.exec_command(f"sudo tc class add dev {interface} parent 1: classid 1:1 htb rate {bandwidth}mbit")
            client.exec_command(f"sudo tc filter add dev {interface} protocol ip parent 1:0 prio 1 u32 match ip dst {dst_node_ip} flowid 1:1")
            logging.info(f'Bandwidth set between {source_node_name} and {dst_node}: {bandwidth} Mbps')
            time.sleep(0.1)  # Short delay to prevent overlap issues

    except Exception as e:
        logging.error(f"Failed to set bandwidth for {source_node_name}: {e}")
    finally:
        client.close()

def automate_bandwidth_injection(params):
    source_node_name, bandwidth_matrix, node_details, node_names = params
    username = node_details[source_node_name]['username']
    key_path = node_details[source_node_name]['key_path']
    interface = 'eth0'  # Assuming the interface name is eth0
    apply_bandwidth_between_nodes(source_node_name, username, key_path, interface, bandwidth_matrix, node_details, node_names)

# Assuming correct IP addresses and no duplication in node keys
node_details = {
    'k8s-worker-1': {'ip': '172.26.128.30', 'username': 'ubuntu', 'key_path': '/home/ubuntu/.ssh/id_rsa'},
    'k8s-worker-2': {'ip': '172.26.132.91', 'username': 'ubuntu', 'key_path': '/home/ubuntu/.ssh/id_rsa'},
    'k8s-worker-3': {'ip': '172.26.133.31', 'username': 'ubuntu', 'key_path': '/home/ubuntu/.ssh/id_rsa'},
    'k8s-worker-4': {'ip': '172.26.132.241', 'username': 'ubuntu', 'key_path': '/home/ubuntu/.ssh/id_rsa'},
    'k8s-worker-5': {'ip': '172.26.132.142', 'username': 'ubuntu', 'key_path': '/home/ubuntu/.ssh/id_rsa'},
    'k8s-worker-6': {'ip': '172.26.133.55', 'username': 'ubuntu', 'key_path': '/home/ubuntu/.ssh/id_rsa'},
    'k8s-worker-7': {'ip': '172.26.130.22', 'username': 'ubuntu', 'key_path': '/home/ubuntu/.ssh/id_rsa'},
    'k8s-worker-8': {'ip': '172.26.130.82', 'username': 'ubuntu', 'key_path': '/home/ubuntu/.ssh/id_rsa'},
    'k8s-worker-9': {'ip': '172.26.133.118', 'username': 'ubuntu', 'key_path': '/home/ubuntu/.ssh/id_rsa'}
}

# Prepare parameters for parallel execution
params_list = [(source_node, bandwidth_matrix, node_details, node_names) for source_node in node_details.keys()]

if __name__ == '__main__':
    with Pool(processes=len(node_details)) as pool:
        pool.map(automate_bandwidth_injection, params_list)
